In [2]:
%pip install openai-whisper

In [3]:
%pip install nemo_toolkit[asr]

In [13]:
%pip install jiwer

In [25]:
# Whisper
import whisper
import time
import wave
from jiwer import wer


model = whisper.load_model("base")

start_time = time.time()


result = model.transcribe("test.wav")

end_time = time.time()


recognition_time = end_time - start_time
print("Распознанный текст:", result["text"])


with wave.open("test.wav", "r") as wf:
    duration = wf.getnframes() / wf.getframerate()

print(f"Длина аудиофайла: {duration:.2f} сек.")
print(f"Время распознавания: {recognition_time:.2f} сек.")


rtf = recognition_time / duration
print(f"RTF (Real-Time Factor): {rtf:.3f}")


reference_text = "1, 2, 3, 1, 2, 3, 1, 2, 3, приём"

error_rate = wer(reference_text, result["text"])
print(f"WER (Word Error Rate): {error_rate:.3%}")


Распознанный текст:  1, 2, 3, 3, 3, 3, 3, 3, 4
Длина аудиофайла: 5.43 сек.
Время распознавания: 185.20 сек.
RTF (Real-Time Factor): 34.084
WER (Word Error Rate): 50.000%


In [23]:
# Jasper
import nemo.collections.asr as nemo_asr
import torch
import torchaudio
import time
from jiwer import wer


model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")


audio_path = "test.wav"


waveform, sample_rate = torchaudio.load(audio_path)


if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)


info = torchaudio.info(audio_path)
duration = info.num_frames / info.sample_rate


start_time = time.time()


transcriptions = model.transcribe([audio_path])

end_time = time.time()


recognition_time = end_time - start_time


print("Jasper Transcription:", transcriptions[0])
print(f"Время распознавания: {recognition_time:.2f} секунд")
print(f"Длина аудиофайла: {duration:.2f} секунд")


rtf = recognition_time / duration
print(f"RTF (Real-Time Factor): {rtf:.3f}")


reference_text = "1,2, 3, 1, 2, 3, 1, 2, 3, приём"

recognized_text = transcriptions[0].text
error_rate = wer(reference_text, recognized_text)

print(f"WER (Word Error Rate): {error_rate:.2%}")


[NeMo I 2025-05-22 14:15:13 nemo_logging:393] Found existing object /root/.cache/torch/NeMo/NeMo_2.3.0/QuartzNet15x5Base-En/b69977d31abaf2ab3e8f082866f6dfcb/QuartzNet15x5Base-En.nemo.
[NeMo I 2025-05-22 14:15:13 nemo_logging:393] Re-using file from: /root/.cache/torch/NeMo/NeMo_2.3.0/QuartzNet15x5Base-En/b69977d31abaf2ab3e8f082866f6dfcb/QuartzNet15x5Base-En.nemo
[NeMo I 2025-05-22 14:15:13 nemo_logging:393] Instantiating model from pre-trained checkpoint
[NeMo I 2025-05-22 14:15:14 nemo_logging:393] PADDING: 16
[NeMo I 2025-05-22 14:15:15 nemo_logging:393] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_2.3.0/QuartzNet15x5Base-En/b69977d31abaf2ab3e8f082866f6dfcb/QuartzNet15x5Base-En.nemo.


Transcribing: 100%|██████████| 1/1 [00:02<00:00,  2.80s/it]

Jasper Transcription: Hypothesis(score=tensor(-18.2510), y_sequence=tensor([28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
        28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
        28, 28, 28, 28, 28, 28, 28, 28, 28,  1, 28, 20, 28, 28, 28, 28, 28, 28,
        28,  0, 28, 28, 22, 28,  5, 28, 28, 28, 28, 28, 28,  0, 28, 28, 28, 20,
         8, 18,  5, 28,  5, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
        28, 28, 28, 28,  0,  0,  0,  0,  0,  0,  0, 28, 28, 28, 28,  8, 28,  1,
        28, 19, 28, 20, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,  1, 28, 18,
        28, 28, 28, 28, 28,  0,  0, 28, 28, 20,  8, 18,  5, 28,  5, 28, 28, 28,
        28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,  0,  0,
        28, 28, 28, 16, 28,  1, 28, 19, 28, 20, 28, 28, 28, 28, 28, 28, 28, 28,
         0,  0, 28, 23, 28,  5, 28, 18, 28, 28, 28, 28, 28,  0,  0, 28, 28, 20,
         8, 18,  5, 28,  5, 28, 28, 28, 28, 28, 28, 

In [6]:
%pip install vosk


In [7]:
import os
import urllib.request
import zipfile

model_url = "https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip"
model_path = "vosk-model-small-en-us-0.15"

if not os.path.exists(model_path):
    print("Загрузка модели...")
    urllib.request.urlretrieve(model_url, "model.zip")
    with zipfile.ZipFile("model.zip", 'r') as zip_ref:
        zip_ref.extractall(model_path)
    print("Модель загружена.")


In [9]:
import zipfile


!wget -O vosk-model-small-en-us-0.15.zip https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip

model_dir = "vosk-model-small-en-us-0.15"

with zipfile.ZipFile("vosk-model-small-en-us-0.15.zip", 'r') as zip_ref:
    zip_ref.extractall(model_dir)


print("Файлы модели:", os.listdir(model_dir))


--2025-05-22 14:05:18--  https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41205931 (39M) [application/zip]
Saving to: ‘vosk-model-small-en-us-0.15.zip’

vosk-model-small-en 100%[===================>]  39.30M  17.1MB/s    in 2.3s    

2025-05-22 14:05:21 (17.1 MB/s) - ‘vosk-model-small-en-us-0.15.zip’ saved [41205931/41205931]

Файлы модели: ['vosk-model-small-en-us-0.15']


In [24]:
# Vosk
import wave
import json
import time
from vosk import Model, KaldiRecognizer
from jiwer import wer


model = Model("vosk-model-small-en-us-0.15/vosk-model-small-en-us-0.15")

audio_path = "test.wav"


with wave.open(audio_path, "rb") as wf:
    duration = wf.getnframes() / wf.getframerate()


start_time = time.time()


with wave.open(audio_path, "rb") as wf:
    rec = KaldiRecognizer(model, wf.getframerate())
    rec.SetWords(True)
    results = []

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            results.append(res)

    final_res = json.loads(rec.FinalResult())
    results.append(final_res)


end_time = time.time()


recognition_time = end_time - start_time


transcript = " ".join([res.get('text', '') for res in results])
print("Распознанный текст:", transcript)


rtf = recognition_time / duration
print(f"Время распознавания (VR): {recognition_time:.2f} секунд")
print(f"Длина аудиофайла: {duration:.2f} секунд")
print(f"RTF (Real-Time Factor): {rtf:.3f}")


reference_text = "1, 2, 3, 1, 2, 3, 1, 2, 3, приём"


error_rate = wer(reference_text, transcript)
print(f"WER (Word Error Rate): {error_rate:.2%}")


Распознанный текст: has got three of has two or three about two or three pretty young
Время распознавания (VR): 1.93 секунд
Длина аудиофайла: 5.43 секунд
RTF (Real-Time Factor): 0.354
WER (Word Error Rate): 140.00%
